In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd

from utils import (
    read_single_problem_from_path_as_adjacency,
    read_single_problem_from_path_as_sparse,
    read_single_problem_from_path_as_sparse_from_adjacency,
    order_nodes_in_descending_order,
    check_validity_for_adjacency,
)
from constants import BASE_INSTANCES_FILES
from heuristics import (
    descending_degree_glutonous_heuristic,
    dynamic_descending_degree_glutonous_heuristic,
    descending_degree_random_heuristic,
    multiple_descending_degree_random_heuristic
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
print(graph)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))


[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]
[1, 4, 6, 9, 7]
Is the clique valid ?  True


In [3]:
# Build clique size and times of execution for most basic heuristic
files = []
instance_reading_times = []
clique_creating_times = []
total_times = []
clique_sizes = []
methods = []

for file in BASE_INSTANCES_FILES:
    for method, verbose_method in [
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ]:
        # Method and file adding
        methods.append(verbose_method)
        files.append(file)

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=BASE_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = method(graph=graph, degrees=degrees)
        clique_sizes.append(len(clique))
        end_of_clique_time = time()
        clique_creating_times.append(end_of_clique_time - end_of_read_time)

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, clique_creating_times, total_times, clique_sizes, methods]:
        display_list.append("/////")


In [4]:
# Display for most basic heuristic

display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "instance time": instance_reading_times,
        "clique time": clique_creating_times,
        "total time": total_times,
    }
)
print(display_dataframe)


              file     method clique size instance time clique time total time
0   brock200_2.col  desc_base           7      0.054014    0.004027   0.058041
1   brock200_2.col   desc_dyn           7      0.051069    0.003986   0.055055
2   brock200_2.col   desc_ran           7      0.054015    0.004947   0.058962
3   brock200_2.col   mult_ran           8         0.054    0.408984   0.462984
4            /////      /////       /////         /////       /////      /////
5    dsjc125.1.col  desc_base           4      0.005032    0.002007   0.007039
6    dsjc125.1.col   desc_dyn           4         0.003    0.002001   0.005001
7    dsjc125.1.col   desc_ran           3      0.004009    0.001991      0.006
8    dsjc125.1.col   mult_ran           3      0.004012    0.185763   0.189775
9            /////      /////       /////         /////       /////      /////
10   random-10.col  desc_base           5           0.0         0.0        0.0
11   random-10.col   desc_dyn           5      0.001